In [2]:
!pip install datasets tqdm pandas

In [3]:
!pip install sentencepiece

In [4]:
!pip install transformers

In [5]:
!pip install wandb


Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper co

In [6]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [7]:
!nvidia-smi

Fri Jan 26 21:34:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import random
import numpy as np
import torch
import datasets

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
df = pd.read_csv('/kaggle/input/80-20ratiofinal/train8020.csv')
df.shape


(20084, 10)

In [12]:
from transformers import (
    T5ForConditionalGeneration, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader


In [13]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)


In [15]:
train_df = pd.read_csv('/kaggle/input/80-20ratiofinal/train8020.csv')
test_df = pd.read_csv('/kaggle/input/80-20ratiofinal/test8020.csv')

In [16]:
model_name = 'csebuetnlp/banglat5_small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [17]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-rqqs20fl
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-rqqs20fl
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=9ecfddab185a7168e73a9b9f4fd02e88b55a3f3f859a448053aada812c61a2c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-vib6l6w2/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=46da4e3697b60f3a31f03b9f34128cf9d18

In [18]:
from normalizer import normalize
train_df['Comment'] = train_df['Comment'].apply(normalize)
train_df['Correct Form'] = train_df['Correct Form'].apply(normalize)
test_df['Comment'] = test_df['Comment'].apply(normalize)
test_df['Correct Form'] = test_df['Correct Form'].apply(normalize)

In [19]:
train_df.shape, test_df.shape

((20084, 10), (5022, 10))

In [20]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

train_df['input_token_len'] = train_df['Comment'].apply(calc_token_len)
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)
test_df.head()

,Video ID,Channel name,Time of Publishing,Title,Genre,Comment,Error,Category,Correct Form,Unnamed: 9,input_token_len
0,62d6PfZxky0,Bangla Bhuter Golpo,2023-01-16T13:00:48Z,Bhuter Cartoon - Railway Station at 2am Night (True Story) Train Horror Story | Bangla Bhuter Golpo,Entertainment,কথা হচ্ছে ওরা এতো ট্রেন পেল কোথায় 🙄 যে এক ঘন্টা পর পর ট্রেন আসতেছে 😳,1,Multiple Errors,কথা হচ্ছে ওরা এত ট্রেন পেল কোথায় 🙄 যে এক ঘণ্টা পর পর ট্রেন আসছে 😳,NaN,20
1,xgMi3vUaw5Y,Baseera,2022-04-02T19:03:14Z,সিরাহ ৭ – অ্যাবিসিনিয়া | Bangla Seerah,Miscellaneous,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন সুনে আমরা আরো বেশি উপকৃর্ত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",1,Spelling,"প্রিয় ভাই আমার,, আপনি আর বেশি বেশি ভিডিও ছাড়েন,, যেন শুনে আমরা আরো বেশি উপকৃত হতে পারি,,, অনেক অপেক্ষায় থাকি।। আল্লাহ জন্য ভালোবাসি আপনাকে।।",NaN,37
2,8Q7mPmy9eTI,BANGLADESHI REACTOR,2023-01-22T07:00:02Z,লাইভে খেজুর পাতার জা*ইঙ্গা বিক্রি করছে টনি আপা | Salman Muqtadir female version | দাদি দাদা কই,Entertainment,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে রিপ্লাই দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",1,Code Switching,"আপু তোমাকে ধন্যবাদ এইভাবে সবাইকে জবাব দিয়ে শিক্ষা দেওয়ার জন্য,, তুমি যদি সুখে থাকতে পারো তাহলে ওদের চুলকায় কেনো।",NaN,24
3,gMeeozaMmIM,Laser Vision,2018-01-13T11:04:17Z,Surja Dighal Bari | Bangla Movie | Dolly Anwar | Zahirul Haque | Rowshan Jamil | Sheikh Niamat Ali,Entertainment,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়ছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",1,Spelling,"এখানে আমার মনে হয় কেউ অভিনয় করে নাই। এটা একদম জীবন এর সাথে মিলে গেছে। একদম নিখুঁত অভিনয়। আগের দিন এর ছবি, নাটক দেখতে সত্যি অনেক সুন্দর লাগে অভিনয় যেন নয় যেন আসল শুধু ক্যামেরা বন্দী করা হয়েছে জীবনটাকে 🙂🙂।খুব সুন্দর লাগল ছবি টা 🙂🙂",NaN,55
4,Vhj-_avEjs0,Ritu Hossain,2023-06-26T12:00:08Z,রিতু নিজের টাকায় রুম ডেকোরেশন করলো | My Home Tour VLOG | Ritu Hossain | Rakib Hossain,Entertainment,ওওআপি আমি তোমার রোম দেখার জন্য অনেক অনেক অনেক হেপি😊,1,Spelling,আপু আমি তোমার কক্ষ দেখার জন্য অনেক অনেক অনেক খুশি😊,NaN,16


In [21]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['Video ID', 'Channel name ', 'Time of Publishing', 'Title', 'Genre', 'Comment', 'Error', 'Category', 'Correct Form', 'Unnamed: 9', 'input_token_len'],
    num_rows: 5022
})

In [24]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['Comment'], example['Correct Form']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs


In [25]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 24
attention_mask 24
labels 24
{'input_ids': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 6673, 2008, 17, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [290, 406, 79, 14, 541, 38, 4576, 2082, 1068, 5, 158, 2630, 16332, 30, 881, 840, 11777, 756, 376, 520, 3157, 20, 2, 1]}


In [26]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=de5c0506c102c3b090d1ddff82ccbd0f7e44978e0b8668e285b269facc091d39
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [27]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [29]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0
    Uninstalling accelerate-0.25.0:
      Successfully uninstalled accelerate-0.25.0


In [31]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=100,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")


In [32]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.328500,1.827368,2.011900,0.807900,2.006200,2.023700,10.906200
1000,1.746600,1.559524,2.360400,0.858500,2.356300,2.369200,11.781600


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars

In [ ]:
trainer.save_model('bangla_gec_model')

In [ ]:


import locale
locale.getpreferredencoding = lambda: "UTF-8"



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences,input_len):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=input_len, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=input_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text


In [ ]:
references,predictions = [],[]
test_d = test_df[test_df['Error']==1]
test_d_sentence = test_d['Comment'].tolist()
test_d_len = test_d['input_token_len'].tolist()
test_d_ground = test_d['Correct Form'].tolist()
import nltk
nltk.download('punkt')
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


In [ ]:
def calculate(reference,hypothesis):
    # Tokenize sentences into words
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)

    # Create n-grams for reference and hypothesis
    reference_1grams = list(ngrams(reference_tokens, 1))
    hypothesis_1grams = list(ngrams(hypothesis_tokens, 1))
    reference_2grams = list(ngrams(reference_tokens, 2))
    hypothesis_2grams = list(ngrams(hypothesis_tokens, 2))

    # Calculate ROUGE scores
    rouge1_precision = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(reference_1grams)
    rouge1_recall = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(hypothesis_1grams)
    rouge2_precision = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(reference_2grams)
    rouge2_recall = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(hypothesis_2grams)

    # Calculate ROUGE-L using NLTK's sentence_bleu function
    smooth = SmoothingFunction().method4
    rougeL = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth)
    d = {
        "rouge1_precision":rouge1_precision,
        "rouge1_recall":rouge1_recall,
        "rouge2_precision":rouge2_precision,
        "rouge2_recall":rouge2_recall,
        "rouge_l":rougeL
    }
    return d

In [ ]:
import nltk
from nltk.util import ngrams

# Function to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores for a pair of texts
def calculate_rouge_scores(reference_tokens, system_tokens):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    # Calculate ROUGE-1 (unigram) scores
    reference_unigrams = set(reference_tokens)
    system_unigrams = set(system_tokens)
    overlap_rouge1 = len(reference_unigrams.intersection(system_unigrams))
    precision_rouge1 = overlap_rouge1 / len(system_unigrams)
    recall_rouge1 = overlap_rouge1 / len(reference_unigrams)
    r1_t = 1 if precision_rouge1 + recall_rouge1 == 0 else 0
    f1_rouge1 = 2 * (precision_rouge1 * recall_rouge1) / (precision_rouge1 + recall_rouge1 + r1_t)

    # Calculate ROUGE-2 (bigram) scores
    reference_bigrams = set(ngrams(reference_tokens, 2))
    system_bigrams = set(ngrams(system_tokens, 2))
    overlap_rouge2 = len(reference_bigrams.intersection(system_bigrams))
    precision_rouge2 = overlap_rouge2 / len(system_bigrams)
    recall_rouge2 = overlap_rouge2 / len(reference_bigrams)
    r2_t = 1 if precision_rouge2 + recall_rouge2 == 0 else 1
    f1_rouge2 = 2 * (precision_rouge2 * recall_rouge2) / (precision_rouge2 + recall_rouge2 + r2_t)

    # Calculate ROUGE-L scores
    lcs_length = lcs(reference_tokens, system_tokens)
    precision_rougeL = lcs_length / len(system_tokens)
    recall_rougeL = lcs_length / len(reference_tokens)
    rL_t = 1 if precision_rougeL + recall_rougeL == 0 else 0
    f1_rougeL = 2 * (precision_rougeL * recall_rougeL) / (precision_rougeL + recall_rougeL + rL_t)

    return {
        'ROUGE-1 Precision': precision_rouge1,
        'ROUGE-1 Recall': recall_rouge1,
        'ROUGE-1 F1': f1_rouge1,
        'ROUGE-2 Precision': precision_rouge2,
        'ROUGE-2 Recall': recall_rouge2,
        'ROUGE-2 F1': f1_rouge2,
        'ROUGE-L Precision': precision_rougeL,
        'ROUGE-L Recall': recall_rougeL,
        'ROUGE-L F1': f1_rougeL,
    }

# Function to calculate the average of ROUGE scores for an array of text pairs
def calculate_average_rouge_scores(reference_texts, system_texts):
    total_scores = {
        'ROUGE-1 Precision': 0,
        'ROUGE-1 Recall': 0,
        'ROUGE-1 F1': 0,
        'ROUGE-2 Precision': 0,
        'ROUGE-2 Recall': 0,
        'ROUGE-2 F1': 0,
        'ROUGE-L Precision': 0,
        'ROUGE-L Recall': 0,
        'ROUGE-L F1': 0,
    }

    num_pairs = len(reference_texts)

    for i in range(num_pairs):
        reference_text = reference_texts[i]
        system_text = system_texts[i]

        reference_tokens = nltk.word_tokenize(reference_text)
        system_tokens = nltk.word_tokenize(system_text)

        scores = calculate_rouge_scores(reference_tokens, system_tokens)

        for key, value in scores.items():
            total_scores[key] += value

    # Calculate the average scores
    average_scores = {key: value / num_pairs for key, value in total_scores.items()}
    
    return average_scores

# Example usage with an array of reference and system texts
reference_texts = test_d_ground
system_texts = [correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0] for i in range(len(test_d_sentence)) ]

average_scores = calculate_average_rouge_scores(reference_texts, system_texts)
print("Average ROUGE Scores:")
for key, value in average_scores.items():
    print(key + ": {:.4f}".format(value))

# #amar lines ============================================

In [ ]:
data = []
for i in range(len(reference_texts)):
    row = [reference_texts[i], system_texts[i]]
    # Add ROUGE scores to the row
    for key, value in scores[i].items():  # Assuming scores is a list of scores for each pair
        row.append(value)
    data.append(row)

# Create the CSV file
with open('rouge_resultsT5sml.csv', 'w', newline='') as csvfile:
    fieldnames = ['Reference Text', 'System Text'] + list(scores[0].keys())  # Use keys from the first score for header
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows({fieldname: value for fieldname, value in row.items()} for row in data)

In [ ]:
# random data

r = train_df['Comment'].tolist()[:20] + test_df['Comment'].tolist()[:20]
r_ln = train_df['input_token_len'].tolist()[:20] + test_df['input_token_len'].tolist()[:20]
reference_texts = r
for i in range(len(r)):
    print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')


In [ ]:
# error data

import time

r = (test_df[test_df['Error']==1])['Comment'].tolist()
r_ln = (test_df[test_df['Error']==1])['input_token_len'].tolist()
reference_texts = r
start_time = time.time()
for i in range(len(r)):
    x = r[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
#     print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')
end_time = time.time()
num_iterations = len(r)
average_inference_time = (end_time - start_time) / num_iterations
print(f"Total Inference Time: {end_time-start_time}")
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

In [ ]:
r_gn = (test_df[test_df['Error']==1])['Correct Form'].tolist()

In [ ]:
predicted_score,predicted_sentences = [],[]
for i in range(len(r_gn)):
    x = r_gn[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
    predicted_sentences.append(y)
    rouge_score_v = calculate_rouge_scores(x, y)
    predicted_score.append(rouge_score_v['ROUGE-L F1'])
# test_df['ROUGE-L F1'] = predicted_sentence



In [ ]:
e_df = test_df[test_df['Error']==1]
e_df['ROUGE-L F1'] = predicted_score
e_df['Predicted Form'] = predicted_sentences


In [ ]:
e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)

In [ ]:
e_df.head(20)[1:]

In [ ]:
e_df.tail(20)